<a href="https://colab.research.google.com/github/Bane-24/Crypto-Price-Prediction/blob/main/Data_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing Libraries
!pip install cryptocmd
!pip install yfinance
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from cryptocmd import CmcScraper
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
# Scrape Bitcoin historical data
scraper = CmcScraper("BTC", "01-01-2014", "31-03-2023")
bitcoin_df = scraper.get_dataframe()

# Scrape stock market data
stock_data = yf.download("SPY", start="2014-01-01", end="2023-03-31")
stock_df = stock_data["Adj Close"].to_frame().reset_index().rename(columns={"Adj Close": "stock_price", "Date": "Date"})

# Merge data and handle NA using interpolation
merged_df = bitcoin_df.merge(stock_df, on="Date", how="inner")
merged_df.set_index("Date", inplace=True)
merged_df.index = pd.to_datetime(merged_df.index)
merged_df.interpolate(method="time", inplace=True)

# Add day, week, and month columns
merged_df["day"] = merged_df.index.day
merged_df["week"] = merged_df.index.week
merged_df["month"] = merged_df.index.month


[*********************100%***********************]  1 of 1 completed


<ipython-input-34-7089b96092ea>:17: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  merged_df["week"] = merged_df.index.week


In [35]:
merged_df.head()

,Open,High,Low,Close,Volume,Market Cap,stock_price,day,week,month
Date,,,,,,,,,,
2023-03-30,28350.141491,29159.902618,27720.159338,28033.563150,2.243535e+10,5.419648e+11,403.700012,30,13,3
2023-03-29,27267.032175,28619.543389,27259.661180,28348.442343,2.068495e+10,5.480255e+11,401.350006,29,13,3
2023-03-28,27132.888921,27460.719459,26677.818728,27268.131483,1.778360e+10,5.271179e+11,395.600006,28,13,3
2023-03-27,27994.067923,28037.925861,26606.689001,27139.888531,1.818890e+10,5.246159e+11,396.489990,27,13,3
2023-03-24,28324.111695,28388.437746,27039.265432,27493.285144,2.598031e+10,5.313710e+11,395.750000,24,12,3


In [36]:
# Daily Change
daily_change = merged_df.copy()

# Calculate the percentage difference of close price between consecutive days
daily_change['pct_change'] = daily_change['Close'].pct_change()
daily_change.iloc[0, daily_change.columns.get_loc("pct_change")] = 0
daily_change.head()

,Open,High,Low,Close,Volume,Market Cap,stock_price,day,week,month,pct_change
Date,,,,,,,,,,,
2023-03-30,28350.141491,29159.902618,27720.159338,28033.563150,2.243535e+10,5.419648e+11,403.700012,30,13,3,0.000000
2023-03-29,27267.032175,28619.543389,27259.661180,28348.442343,2.068495e+10,5.480255e+11,401.350006,29,13,3,0.011232
2023-03-28,27132.888921,27460.719459,26677.818728,27268.131483,1.778360e+10,5.271179e+11,395.600006,28,13,3,-0.038108
2023-03-27,27994.067923,28037.925861,26606.689001,27139.888531,1.818890e+10,5.246159e+11,396.489990,27,13,3,-0.004703
2023-03-24,28324.111695,28388.437746,27039.265432,27493.285144,2.598031e+10,5.313710e+11,395.750000,24,12,3,0.013021


In [37]:
# Weekly Change
weekly_change = merged_df.copy()

# Group the rows by week and calculate the weekly average
weekly_change = weekly_change.groupby('week')['Open', 'High', 'Low', 'Close', 'Volume','Market Cap', 'stock_price'].mean().reset_index()

# Calculate the percentage difference of close price between consecutive weeks
weekly_change['pct_change'] = weekly_change['Close'].pct_change()
weekly_change.iloc[0, weekly_change.columns.get_loc("pct_change")] = 0
weekly_change.head()


<ipython-input-37-6f28fa93c8db>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weekly_change = weekly_change.groupby('week')['Open', 'High', 'Low', 'Close', 'Volume','Market Cap', 'stock_price'].mean().reset_index()


,week,Open,High,Low,Close,Volume,Market Cap,stock_price,pct_change
0,1,14182.688335,14690.735013,13675.642603,14330.276370,2.140178e+10,2.649055e+11,280.401000,0.000000
1,2,12636.972508,13008.557726,12042.082520,12641.873270,1.855276e+10,2.333060e+11,271.666730,-0.117821
2,3,11580.595299,11875.577535,10980.228595,11370.427838,1.722188e+10,2.102240e+11,266.406509,-0.100574
3,4,12523.273569,12958.891219,12089.044045,12572.128558,1.798598e+10,2.331890e+11,275.764583,0.105686
4,5,12209.851237,12570.313364,11925.426276,12331.481512,1.581167e+10,2.288466e+11,273.166404,-0.019141


In [38]:
# Monthly Change
monthly_change = merged_df.copy()

# Group the rows by month and calculate the monthly average
monthly_change = monthly_change.groupby('month')['Open', 'High', 'Low', 'Close', 'Volume','Market Cap', 'stock_price'].mean().reset_index()

# Calculate the percentage difference of close price between consecutive months
monthly_change['pct_change'] = monthly_change['Close'].pct_change()
monthly_change.iloc[0, monthly_change.columns.get_loc("pct_change")] = 0
monthly_change.head()

<ipython-input-38-1d0c34f484aa>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  monthly_change = monthly_change.groupby('month')['Open', 'High', 'Low', 'Close', 'Volume','Market Cap', 'stock_price'].mean().reset_index()


,month,Open,High,Low,Close,Volume,Market Cap,stock_price,pct_change
0,1,12198.138200,12570.928082,11693.048872,12191.350138,1.790514e+10,2.252225e+11,270.772616,0.000000
1,2,13350.422878,13803.085935,12940.444379,13452.058816,2.007223e+10,2.501152e+11,273.228934,0.103410
2,3,14781.531610,15199.286728,14366.894311,14825.821696,1.807855e+10,2.766234e+11,270.767366,0.102123
3,4,13470.673491,13784.899356,13132.686890,13484.567685,1.756570e+10,2.500927e+11,259.940721,-0.090467
4,5,11756.451649,12112.896291,11231.751891,11623.484615,1.993434e+10,2.146908e+11,258.782535,-0.138016
